# SVM Regression

SVM algorithm is quite versatile: not only does it support linear and nonlinear classification, but it also supports linear and nonlinear regression. The trick is to reverse the objective: instead of trying to fit the largest possible street between two classes while limiting margin violations, , SVM Regression tries to fit as many instances as possible on the street while limiting margin violations (i.e., instances off the street). The width of the street is controlled by a hyperparameter ϵ. Figure shows two linear SVM Regression models trained on some random linear data, one with a large margin (ϵ = 1.5) and the other with a small margin (ϵ = 0.5).

<img src='images/img10.PNG'>

Adding more training instances within the margin does not affect the model’s predictions; thus, the model is said to be ϵ insensitive. 

Scikit-Learn provide LinearSVR class to perform linear SVM Regression.

In [1]:
from sklearn.datasets import load_iris
import numpy as np

iris=load_iris()
X=iris["data"][:,(2,3)]
y=(iris["target"]==2).astype(np.float64)

In [2]:
from sklearn.svm import LinearSVR

svm_reg=LinearSVR(epsilon=1.5)
svm_reg.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

To tackle nonlinear regression tasks, you can use a kernelized SVM model.
For example, below figure shows SVM Regression on a random quadratic training set, using a 2nd-degree polynomial kernel. There is little regularization on the left plot (i.e., a large C value), and much more regularization on the right plot (i.e., a small C value).

<img src='images/img11.PNG'>

The following code produces the model represented on the left of above figure using Scikit-Learn’s SVR class (which supports the kernel trick). The SVR class is the regression equivalent of the SVC class, and the LinearSVR class is the regression equivalent of the LinearSVC class. The LinearSVR class scales linearly with the size of the training set (just like the LinearSVC class), while the SVR class gets much too slow when the training set grows large (just like the SVC class). 

In [3]:
from sklearn.svm import SVR

svm_poly_reg=SVR(kernel='poly',degree=2,C=100,epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# Behind the scene

This section explains how SVMs make predictions and how their training algorithms work, starting with linear SVM classifiers. You can safely skip it and go straight to the exercises at the end of this chapter if you are just getting started with Machine Learning, and come back later when you want to get a deeper understanding of SVMs.

### Decision Function and Predictions

The linear SVM classifier model predicts the class of a new instance x by simply computing the decision function wT · x + b = w1 x1 + ⋯ + wn xn + b: if the result is positive, the predicted class ŷ is the positive class (1), or else it is the negative class (0); 

<img src='images/img12.PNG'>

it is a two-dimensional plane since this dataset has two features (petal width and petal length). The decision boundary is the set of points where the decision function is equal to 0: it is the intersection of two planes, which is a straight line (represented by the thick solid line).

<img src='images/img13.PNG'>

The dashed lines represent the points where the decision function is equal to 1 or –1: they are parallel and at equal distance to the decision boundary, forming a margin around it. Training a linear SVM classifier means finding the value of w and b that make this margin as wide as possible while avoiding margin violations (hard margin) or limiting them (soft margin).

### Training Objective

Consider the slope of the decision function: it is equal to the norm of the weight vector, ∥ w ∥. If we divide this slope by 2, the points where the decision function is equal to ±1 are going to be twice as far away from the decision boundary. In other words, dividing the slope by 2 will multiply the margin by 2. Perhaps this is easier to visualize in 2D. The smaller the weight vector w, the larger the margin.

<img src='images/img14.PNG'>

So we want to minimize ∥ w ∥ to get a large margin. However, if we also want to avoid any margin violation (hard margin), then we need the decision function to be greater than 1 for all positive training instances, and lower than –1 for negative training instances. If we define t(i) = –1 for negative instances (if y(i) = 0) and t(i) = 1 for positive instances (if y(i) = 1), then we can express this constraint as t(i)(wT · x(i) + b) ≥ 1 for all instances. 

We can therefore express the hard margin linear SVM classifier objective as the constrained optimization problem in below equation:

<img src='images/img15.PNG'>

# Quadratic Programming

The hard margin and soft margin problems are both convex quadratic optimization problems with linear constraints. Such problems are known as Quadratic Programming (QP) problems.

<img src='images/img16.PNG'>

Note that the expression A · p ≤ b actually defines nc constraints: pT · a(i) ≤ b(i) for i = 1, 2, ⋯, nc, where a(i) is the vector containing the elements of the ith row of A and b(i) is the ith element of b. 

You can easily verify that if you set the QP parameters in the following way, you get the hard margin linear SVM classifier objective: 

    * np = n + 1, where n is the number of features (the +1 is for the bias term). 
    * nc = m, where m is the number of training instances. 
    * H is the np × np identity matrix, except with a zero in the top-left cell (to ignore the bias term). 
    * f = 0, an np-dimensional vector full of 0s. 
    * b = 1, an nc-dimensional vector full of 1s. 
    * a(i) = –t(i) (i), where (i) is equal to x(i) with an extra bias feature 0 = 1.
    
So one way to train a hard margin linear SVM classifier is just to use an off-the-shelf QP solver by passing it the preceding parameters. The resulting vector p will contain the bias term b = p0 and the feature weights wi = pi for i = 1, 2, ⋯, m. Similarly, you can use a QP solver to solve the soft margin problem (see the exercises at the end of the chapter). However, to use the kernel trick we are going to look at a different constrained optimization problem.